In [1]:
import pandas as pd
import re
import os
import locale
from datetime import datetime
from dateutil import parser

In [24]:
#data_es = pd.read_csv('/Users/paulinesmirnova/Documents/GitHub/superlexis/data/ES_DATA.csv')
#data_uk = pd.read_csv('/Users/paulinesmirnova/Documents/GitHub/superlexis/data/UK_DATA.csv')
#data_fr = pd.read_csv('/Users/paulinesmirnova/Documents/GitHub/superlexis/data/FR_DATA.csv')
#data_nl = pd.read_csv('/Users/paulinesmirnova/Documents/GitHub/superlexis/data/NL_data_translated.csv.csv')
data = pd.read_csv('/Users/paulinesmirnova/Documents/GitHub/superlexis/supreme_data.csv')
data.head()

,Unnamed: 0,headline,news_source,datePublished,region,articleSection,length,byline,text,highlight,translated,country,datePublished_clean
0,1,Het Lombokplein wordt te gevaarlijk'; Bus- en ...,de telegraaf,"April 19, 2023 Wednesday",Utrecht,REGIO; Blz. 12,423 words,Michiel,Bus- en tramkruising terug naar tekentafel doo...,NaN,Bus and tram intersection back to drawing boar...,NL,19/04/2023
1,2,Utrecht is echt een fietshemel'; Route Vuelta ...,de telegraaf,"April 20, 2023 Thursday",Nederland,BINNENLAND; Blz. 12,276 words,Eric Roeske,Route Vuelta ligt er strak bij door Eric Roe...,NaN,Route Vuelta is in good shape by Eric Roeske U...,NL,20/04/2023
2,4,"Als de auto wegvalt, crasht de economie'; Bran...",de telegraaf,"April 21, 2023 Friday",Haaglanden,REGIO; Blz. 13,447 words,Ingrid de Groot,Brandbrief honderden ondernemers over plannen ...,NaN,Urgent letter to hundreds of entrepreneurs abo...,NL,21/04/2023
3,5,Beseffen jullie wel hoe perféct Amsterdam is?'...,nrc handelsblad,"April 22, 2023 Saturday",1ste Editie,AMSTERDAM; Blz. 1,1992 words,Stan Putman Foto Simon Lenskens,ABSTRACT Not Just Bikes De filmpjes die de Can...,NaN,ABSTRACT Not Just Bikes The videos that Canadi...,NL,22/04/2023
4,6,Bewijs dat betaalbare woning nog steeds kan'; ...,de telegraaf,"April 23, 2023 Sunday",Nederland,FINANCIEEL; Blz. 25,627 words,Yteke de Jong,Aannemer Van Wijnen schroeft bouwproductie op ...,NaN,Contractor Van Wijnen increases construction p...,NL,23/04/2023


In [5]:
# Convert all text in all columns to lowercase
def to_lowercase(text):
    if isinstance(text, str):
        return text.lower()
    return text

# Apply the lowercase conversion to all columns
data = data.applymap(to_lowercase)

/var/folders/98/bbr4z_5s1yj02ygmn84xmtwc0000gn/T/ipykernel_10753/2740884162.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(to_lowercase)


## Date function for English language

In [13]:
def extract_and_format_date(date_str):
    try:
        date_obj = parser.parse(date_str, fuzzy=True)
        return date_obj.strftime("%d/%m/%Y")
    except Exception as e:
        print(f"Error parsing date '{date_str}': {e}")
        return None

# Function to apply to each row in the DataFrame
def extract_and_format_dates_in_df(df, column_name):
    df['date'] = df[column_name].apply(extract_and_format_date)
    return df

# Apply the function to create a new column 'formatted_date'
data = extract_and_format_dates_in_df(data, 'datePublished')

In [15]:
data.date

0       19/04/2023
1       20/04/2023
2       21/04/2023
3       22/04/2023
4       23/04/2023
           ...    
5360    28/12/2016
5361    06/03/2009
5362    22/01/2014
5363    15/08/2018
5364    07/07/2016
Name: date, Length: 5365, dtype: object

In [17]:
data.date.isna().sum()

0

In [19]:
data.date.value_counts()

date
21/12/2023    13
23/02/2024    11
01/06/2024    10
13/04/2024    10
08/09/2023     9
              ..
03/09/2015     1
07/05/2020     1
05/03/2020     1
19/08/2021     1
06/03/2009     1
Name: count, Length: 2928, dtype: int64

In [21]:
data

,Unnamed: 0,headline,news_source,datePublished,region,articleSection,length,byline,text,highlight,translated,country,formatted_date,date
0,1,het lombokplein wordt te gevaarlijk'; bus- en ...,de telegraaf,"april 19, 2023 wednesday",utrecht,regio; blz. 12,423 words,michiel,bus- en tramkruising terug naar tekentafel doo...,NaN,bus and tram intersection back to drawing boar...,nl,19/04/2023,19/04/2023
1,2,utrecht is echt een fietshemel'; route vuelta ...,de telegraaf,"april 20, 2023 thursday",nederland,binnenland; blz. 12,276 words,eric roeske,route vuelta ligt er strak bij door eric roe...,NaN,route vuelta is in good shape by eric roeske u...,nl,20/04/2023,20/04/2023
2,4,"als de auto wegvalt, crasht de economie'; bran...",de telegraaf,"april 21, 2023 friday",haaglanden,regio; blz. 13,447 words,ingrid de groot,brandbrief honderden ondernemers over plannen ...,NaN,urgent letter to hundreds of entrepreneurs abo...,nl,21/04/2023,21/04/2023
3,5,beseffen jullie wel hoe perféct amsterdam is?'...,nrc handelsblad,"april 22, 2023 saturday",1ste editie,amsterdam; blz. 1,1992 words,stan putman foto simon lenskens,abstract not just bikes de filmpjes die de can...,NaN,abstract not just bikes the videos that canadi...,nl,22/04/2023,22/04/2023
4,6,bewijs dat betaalbare woning nog steeds kan'; ...,de telegraaf,"april 23, 2023 sunday",nederland,financieel; blz. 25,627 words,yteke de jong,aannemer van wijnen schroeft bouwproductie op ...,NaN,contractor van wijnen increases construction p...,nl,23/04/2023,23/04/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5360,5431,mi dica per quale partito ha votato; il sindac...,italiaoggi,"december 28, 2016 wednesday",NaN,primo piano; pag. 6,1081 words,carlo valentini twitter: @cavalent,bando pubblico: chi vuole essere assunto (per ...,la difesa: il prescelto deve avere la mia fiducia,public announcement: who wants to be hired (fo...,it,28/12/2016,28/12/2016
5361,5432,"multiutility sì, ma integrata; le aziende loca...",italiaoggi,"march 6, 2009 friday",NaN,"rapporto belluno, treviso e vicenza; pag. 25",849 words,samuele nottegar,"acqua, rifiuti, energia: in quale direzione si...","le società che si muovono tra acqua, rifiuti, ...","water, waste, energy: in which direction are v...",it,06/03/2009,06/03/2009
5362,5433,risiko agricolo,italiaoggi,"january 22, 2014 wednesday",NaN,attualità; pag. 36,926 words,NaN,granarolo entra in un segmento strategico e in...,NaN,granarolo enters a strategic and rapidly growi...,it,22/01/2014,22/01/2014
5363,5434,commenti,italiaoggi,"august 15, 2018 wednesday",NaN,primo piano; pag. 11,676 words,NaN,in italia l'unico ponte che non crolla è quell...,NaN,in italy the only bridge that doesn't collapse...,it,15/08/2018,15/08/2018
